In [1]:
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from dataclasses import dataclass

In [2]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


### Prepare the data

In [4]:
class ImpurityDataset(Dataset):
    
    def __init__(self, dataframe, fixed_features, labels, feature_scaler=None, label_scaler=None, device=None):
        assert len(labels) % 2 == 0
        
        self.fixed_features = fixed_features
        self.labels = labels
        self.n_samples = len(dataframe)
        
        self.output_length = 2
        self.input_length = len(fixed_features) + len(labels) - self.output_length
        self.sequence_length = len(labels) // self.output_length

        df_features = dataframe[fixed_features]
        df_labels = dataframe[labels]

        if feature_scaler is not None and label_scaler is not None:
            xs = feature_scaler.transform(df_features.values)
            ys = label_scaler.transform(df_labels.values)
        else:
            xs = df_features.values
            ys = df_labels.values

        feature_data = np.zeros((self.n_samples, self.sequence_length, self.input_length))
        label_data = np.zeros((self.n_samples, self.sequence_length, self.output_length))
        
        for i in range(self.n_samples):
            for j in range(self.sequence_length):
                xi = xs[i]
                xj = ys[i][0:j*2]
                y = ys[i][j*2:j*2+2]
        
                features = np.concatenate([xi, xj], axis=0)
                pad_width = self.input_length - len(features)
                feature_data[i, j, :] = np.pad(features, (0, pad_width))
                label_data[i, j, :] = y

        self.feature_data = torch.tensor(feature_data, dtype=torch.float).to(device)
        self.label_data = torch.tensor(label_data, dtype=torch.float).to(device)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

In [5]:
def compute_scalers(dataframe, fixed_features, labels, test_size=0.1, random_state=None):
    train_df, _ = train_test_split(dataframe, test_size=test_size, random_state=random_state)
    df_features = train_df[fixed_features]
    df_labels = train_df[labels]
    
    feature_scaler = StandardScaler()
    feature_scaler.fit(df_features.values)
    
    label_scaler = StandardScaler()
    label_scaler.fit(df_labels.values) 

    return feature_scaler, label_scaler      

In [27]:
file_path = '../data/20230825_144318_10k_EVDoubExp-TExp-wmax5-sparse-hyb_with_perturbation.csv'

#fixed_features = ['beta', 'U', 'Eimp', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']
fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']
#fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3', 'ReFso1', 'ImFso1', 'ReFso3', 'ImFso3', 'ReFso5', 'ImFso5', 'ReFso7', 'ImFso7', 'ReFso9', 'ImFso9', 'ReFso11', 'ImFso11', 'ReFso13', 'ImFso13', 'ReFso15', 'ImFso15', 'ReFso17', 'ImFso17', 'ReFso19', 'ImFso19', 'ReFso21', 'ImFso21', 'ReFso23', 'ImFso23', 'ReFso25', 'ImFso25', 'ReFso29', 'ImFso29', 'ReFso33', 'ImFso33', 'ReFso37', 'ImFso37', 'ReFso43', 'ImFso43', 'ReFso49', 'ImFso49', 'ReFso57', 'ImFso57', 'ReFso69', 'ImFso69', 'ReFso83', 'ImFso83', 'ReFso101', 'ImFso101', 'ReFso127', 'ImFso127', 'ReFso165', 'ImFso165', 'ReFso237', 'ImFso237', 'ReFso399', 'ImFso399', 'ReFso1207', 'ImFso1207']
labels = ['ReSf1', 'ImSf1', 'ReSf3', 'ImSf3', 'ReSf5', 'ImSf5', 'ReSf7', 'ImSf7', 'ReSf9', 'ImSf9', 'ReSf11', 'ImSf11', 'ReSf13', 'ImSf13', 'ReSf15', 'ImSf15', 'ReSf17', 'ImSf17', 'ReSf19', 'ImSf19', 'ReSf21', 'ImSf21', 'ReSf23', 'ImSf23', 'ReSf25', 'ImSf25', 'ReSf29', 'ImSf29', 'ReSf33', 'ImSf33', 'ReSf37', 'ImSf37', 'ReSf43', 'ImSf43', 'ReSf49', 'ImSf49', 'ReSf57', 'ImSf57', 'ReSf69', 'ImSf69', 'ReSf83', 'ImSf83', 'ReSf101', 'ImSf101', 'ReSf127', 'ImSf127', 'ReSf165', 'ImSf165', 'ReSf237', 'ImSf237', 'ReSf399', 'ImSf399', 'ReSf1207', 'ImSf1207']
#labels = ['ReSf1207', 'ImSf1207', 'ReSf399', 'ImSf399', 'ReSf237', 'ImSf237', 'ReSf165', 'ImSf165', 'ReSf127', 'ImSf127', 'ReSf101', 'ImSf101', 'ReSf83', 'ImSf83', 'ReSf69', 'ImSf69', 'ReSf57', 'ImSf57', 'ReSf49', 'ImSf49', 'ReSf43', 'ImSf43', 'ReSf37', 'ImSf37', 'ReSf33', 'ImSf33', 'ReSf29', 'ImSf29', 'ReSf25', 'ImSf25', 'ReSf23', 'ImSf23', 'ReSf21', 'ImSf21', 'ReSf19', 'ImSf19', 'ReSf17', 'ImSf17', 'ReSf15', 'ImSf15', 'ReSf13', 'ImSf13', 'ReSf11', 'ImSf11', 'ReSf9', 'ImSf9', 'ReSf7', 'ImSf7', 'ReSf5', 'ImSf5', 'ReSf3', 'ImSf3', 'ReSf1', 'ImSf1']

df = pd.read_csv(file_path, skiprows=4) # we skip the first four lines, because they are just metadata
df = df[fixed_features + labels]

# remove one special row, looks very weird; ReSf1 = 2.377167465976437e-06
df = df[df['ReSf1'] >= 1e-05]

validation_size = 0.1 # 90% training, 10% for validation

use_scaling = True

if use_scaling:
    feature_scaler, label_scaler = compute_scalers(df, fixed_features, labels, validation_size, seed) # make sure we use the same seed, otherwise the two splits differ!
    dataset = ImpurityDataset(df, fixed_features, labels, feature_scaler, label_scaler, device)         
else:
    dataset = ImpurityDataset(df, fixed_features, labels, device=device)

indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=validation_size, random_state=seed)  # make sure we use the same seed, otherwise the two splits differ!

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)

In [28]:
dataset.__getitem__(0)

(tensor([[ 1.7253, -0.2833, -0.1903,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.7253, -0.2833, -0.1903,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.7253, -0.2833, -0.1903,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 1.7253, -0.2833, -0.1903,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.7253, -0.2833, -0.1903,  ..., -1.9985,  0.0000,  0.0000],
         [ 1.7253, -0.2833, -0.1903,  ..., -1.9985,  0.0287, -1.9653]],
        device='cuda:0'),
 tensor([[ 1.4672e-01,  6.0201e-01],
         [ 6.0624e-02,  8.5195e-01],
         [ 2.4445e-02,  9.2148e-01],
         [ 4.7469e-03,  8.9540e-01],
         [-7.2976e-03,  8.0284e-01],
         [-1.5029e-02,  6.6537e-01],
         [-2.0027e-02,  5.0002e-01],
         [-2.3156e-02,  3.1974e-01],
         [-2.4946e-02,  1.3394e-01],
         [-2.5747e-02, -5.0702e-02],
         [-2.5808e-02, -2.2963e-01],
         [-2.5312e-02, -3.9989e-01],
         [-2.4399e-02, -5.5967e-01],
         [-2.1736e-02, -8.4473e-01],
         [-1.84

### Define the model

In [29]:
class PositionalEncodingL(nn.Module):
    def __init__(self, T, C, dropout):
        super(PositionalEncodingL, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.positional_embedding = nn.Parameter(torch.zeros(T, C))
    
    def forward(self, x):
        B, T, C = x.shape
        position_encoded = self.positional_embedding[:T, :].unsqueeze(0).expand(B, -1, -1)
        x = x + position_encoded
        return self.dropout(x)

In [57]:
@dataclass
class ModelConfig:
    input_dim: int
    output_dim: int
    sequence_length: int
    
    d_model: int
    nhead: int
    num_layers: int
    dim_feedforward: int
    
    dropout: float
    activation: str
    bias: bool

class AutoregressiveTransformer(nn.Module):
    
    def __init__(self, config, device):
        super(AutoregressiveTransformer, self).__init__()

        self.config = config

        self.input_projection = nn.Linear(config.input_dim, config.d_model)
        self.positional_encoding = PositionalEncodingL(config.sequence_length, config.d_model, config.dropout)
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.d_model, 
            nhead=config.nhead, 
            dim_feedforward=config.dim_feedforward, 
            dropout=config.dropout,
            activation=config.activation, 
            batch_first=True, 
            norm_first=True, 
            bias=config.bias
        )
        
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.num_layers)
        self.sl1 = nn.Linear(config.d_model, config.d_model)
        self.gelu = nn.GELU()
        self.output_layer = nn.Linear(config.d_model, config.output_dim)

        self.att_mask = {}
        
    def forward(self, x):        
        x = self.input_projection(x)        
        x = self.positional_encoding(x)

        seq_len = x.size(1)
        if seq_len not in self.att_mask:
            self.att_mask[seq_len] = self.generate_mask(seq_len, device)
        
        output = self.transformer_decoder(x, x, tgt_mask=self.att_mask[seq_len])
        output = self.sl1(output)
        output = self.gelu(output)
        output = self.output_layer(output)
        
        return output

    def generate_mask(self, sequence_length, device):
        mask = torch.zeros((sequence_length, sequence_length), device=device)
        mask = torch.triu(mask.fill_(float('-inf')), diagonal=1)
            
        return mask

### Initialize the model

In [62]:
config = ModelConfig(
    input_dim = len(fixed_features) + len(labels) - 2,
    output_dim = 2,
    sequence_length = 27,
    
    d_model = 128,
    nhead = 4,
    num_layers = 4,
    dim_feedforward = 128 * 4,
    
    dropout = 0.2,
    activation = 'gelu',
    bias = True
)

model = AutoregressiveTransformer(config, device).to(device)
print(sum(p.numel() for p in model.parameters())/1e3, 'k parameters')
criterion = nn.MSELoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

1086.21 k parameters


In [63]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    
    for inputs, targets in train_loader:

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

In [64]:
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():

        for inputs, targets in val_loader:
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
    
    return total_loss / len(val_loader)

### Train the model

In [65]:
num_epochs = 200

for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion, device)
    train_loss = validate(model, train_loader, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    train_mape = validate_mape(model, train_loader, device, use_scaling=use_scaling)
    val_mape = validate_mape(model, val_loader, device, use_scaling=use_scaling)

    if epoch%5 == 0:
        val_reg = autoregressive_error(100, 27, 0, len(fixed_features), len(fixed_features) + len(labels) - 2, val_loader, use_scaling)
        print(f"Epoch {(epoch+1):3d}: Train Loss: {train_loss:.6f}, Train MAPE: {train_mape:.3f}%, Val Loss: {val_loss:.6f}, Val MAPE: {val_mape:.3f}%, Val REG: {val_reg:.3f}%")
        if val_reg <= 4:
            break
    else:
        print(f"Epoch {(epoch+1):3d}: Train Loss: {train_loss:.6f}, Train MAPE: {train_mape:.3f}%, Val Loss: {val_loss:.6f}, Val MAPE: {val_mape:.3f}%")

Epoch   1: Train Loss: 0.034187, Train MAPE: 31.669%, Val Loss: 0.036533, Val MAPE: 39.508%, Val REG: 70.820%
Epoch   2: Train Loss: 0.022160, Train MAPE: 26.583%, Val Loss: 0.026411, Val MAPE: 33.711%
Epoch   3: Train Loss: 0.014780, Train MAPE: 20.774%, Val Loss: 0.021899, Val MAPE: 25.530%
Epoch   4: Train Loss: 0.010044, Train MAPE: 17.877%, Val Loss: 0.011229, Val MAPE: 21.870%
Epoch   5: Train Loss: 0.010025, Train MAPE: 13.797%, Val Loss: 0.012031, Val MAPE: 16.572%
Epoch   6: Train Loss: 0.007237, Train MAPE: 13.066%, Val Loss: 0.008097, Val MAPE: 15.814%, Val REG: 52.870%
Epoch   7: Train Loss: 0.006459, Train MAPE: 10.795%, Val Loss: 0.007004, Val MAPE: 13.067%
Epoch   8: Train Loss: 0.006304, Train MAPE: 9.928%, Val Loss: 0.006920, Val MAPE: 11.909%
Epoch   9: Train Loss: 0.004215, Train MAPE: 8.378%, Val Loss: 0.004761, Val MAPE: 10.091%
Epoch  10: Train Loss: 0.004700, Train MAPE: 7.788%, Val Loss: 0.006586, Val MAPE: 9.585%
Epoch  11: Train Loss: 0.003615, Train MAPE: 7.1

### Manual model validation

In [66]:
def validate_mape(model, loader, device, use_scaling=use_scaling, epsilon=1e-8):
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        
        for inputs, targets in loader:
            
            outputs = model(inputs)

            if use_scaling:
                outputs = torch.tensor(reverse_tranform_output(outputs, label_scaler))
                targets = torch.tensor(reverse_tranform_output(targets, label_scaler))
            
            ape = torch.abs((targets - outputs) / (targets + epsilon))
            mape = torch.mean(ape) * 100
                    
            total_loss += mape.item()
    
    return total_loss / len(loader)

In [67]:
def reverse_tranform_output(data, scaler):
    B, T, C = data.shape
    data = data.view(B, T*C)
    data = data.cpu().numpy()
    return scaler.inverse_transform(data)

In [68]:
validate_mape(model, val_loader, device, use_scaling=use_scaling)

1.9841176383197308

### Sample from the model

In [69]:
def sample_from_model(model, input, sequence_length, fixed_feature_len, fixed_labels_len, max_features, device):
    model.eval()

    fixed_features = input[:fixed_feature_len]
    fixed_labels = input[fixed_feature_len:fixed_labels_len*2 + fixed_feature_len]
    
    assert len(fixed_labels) % 2 == 0
    provided_sequences = len(fixed_labels) // 2
    
    initial_input = torch.zeros(1, provided_sequences+1, max_features)
    initial_input[0, 0, :fixed_feature_len] = fixed_features

    outputs = torch.zeros(sequence_length, 2, device = device)
    
    for i in range(1, provided_sequences + 1):
        labels = fixed_labels[(i-1)*2:2*i]
        initial_input[0, i, :2*i + fixed_feature_len] = initial_input[0, i-1, :2*i + fixed_feature_len]
        initial_input[0, i, 2*(i-1) + fixed_feature_len:2*i + fixed_feature_len] = labels
        outputs[i-1, :] = labels

    current_input = initial_input.to(device)
    
    with torch.no_grad():
    
        for i in range(fixed_labels_len + 1, sequence_length + 1):

            output = model(current_input)
            predictions = output[0, -1, :]
            
            outputs[i-1, :] = predictions
            
            if i == sequence_length:
                break
            
            position_to_insert = i * 2 + fixed_feature_len
            temp = torch.zeros(1, 1, max_features, device=device)
            temp[0, 0, :position_to_insert-2] = current_input[0, i-1, :position_to_insert-2]
            temp[0, 0, position_to_insert-2:position_to_insert] = predictions
            current_input = torch.cat((current_input, temp), dim=1)

    return outputs

In [70]:
def calculate_mape(true_values, predictions, start, epsilon=1e-8):
    true_values = true_values[:, start:]
    predictions = predictions[:, start:]
    
    mape = torch.mean(torch.abs((true_values - predictions) / (true_values + epsilon)))* 100
    return mape.item()

In [71]:
def autoregressive_error(n_samples, sequence_length, fixed_labels_len, fixed_features_len, max_features, loader, use_scaling):
    mapes = []
    
    for idx in range(n_samples):
        input = loader.dataset[idx][0][fixed_labels_len]
        
        outputs = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
        targets = loader.dataset[idx][1]
    
        T, C = outputs.shape
        outputs = outputs.view(1, T, C)
        targets = targets.view(1, T, C)
        
        if use_scaling:
            outputs = torch.tensor(reverse_tranform_output(outputs, label_scaler))
            targets = torch.tensor(reverse_tranform_output(targets, label_scaler))
        
        mapes.append(calculate_mape(targets, outputs, fixed_labels_len))

    return np.mean(mapes)

In [75]:
n_samples = 100
sequence_length = 27
fixed_labels_len = 15
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2
loader = val_loader

autoregressive_error(n_samples, sequence_length, fixed_labels_len, fixed_features_len, max_features, loader, use_scaling)

5.019063957929611

In [82]:
n_samples = len(val_loader.dataset)
sequence_length = 27
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2
loader = val_loader

total_mapes = []

for j in range(sequence_length):
    mape = autoregressive_error(n_samples, sequence_length, j, fixed_features_len, max_features, loader, use_scaling)
    total_mapes.append(mape)
    print(total_mapes)

print("final output")
print(total_mapes)

[45.941008078500296]
[45.941008078500296, 39.60480784218556]
[45.941008078500296, 39.60480784218556, 31.571197920881765]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734, 23.945855725938884]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734, 23.945855725938884, 21.818009250360717]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734, 23.945855725938884, 21.818009250360717, 19.76058903054692]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734, 23.945855725938884, 21.818009250360717, 19.76058903054692, 17.23577772023932]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734, 23.945855725938884, 21.818009250360717, 19.76058903054692, 17.23577772023932, 15.24018933175795]
[45.941008078500296, 39.60480784218556, 31.571197920881765, 26.786610456299734, 23.945855725938884, 21.81